In [1]:
using DifferentialEquations
using DiffEqGPU
using JLD2
using LinearAlgebra
using Plots
using Roots
using StaticArrays
using Statistics

include("../model/GPUPlant.jl")

In [ ]:
state = GPUPlant.default_state
u0 = state

In [ ]:
# The function which must be minimized to find the equilibrium voltage.
function Ca_difference(p, v)
    IKCa = p.gI*Plant.h∞(v)*Plant.m∞(v)^3*(p.EI-v) + p.gK*Plant.n∞(v)^4*(p.EK-v) + p.gT*Plant.x∞(p, v)*(p.EI-v) + p.gₕ*(p.Eₕ-v)/((1+exp(10*(v-50)))*(1+exp(-(v-63)/7.8))^3) + p.gL*(p.EL-v)
    x_null_Ca = 0.5*IKCa/(p.gKCa*(v-p.EK) - IKCa)
    Ca_null_Ca = p.Kc*Plant.x∞(p, v)*(p.ECa-v+p.ΔCa)
    return x_null_Ca - Ca_null_Ca
end

In [ ]:
# Finds the equilibrium in the slow subsystem.
function Ca_x_eq(p, min_V=(p.EL+p.EK)/2)
    v_eq = find_zero(v -> Ca_difference(p, v), min_V)
    Ca_eq = p.Kc*Plant.x∞(p, v_eq)*(p.ECa-v_eq+p.ΔCa)
    x_eq = Plant.x∞(p, v_eq)
    return v_eq, Ca_eq, x_eq
end

In [ ]:
function countSpikes(sol, p, debug=false)
    # Obtain burst reset times.
    resets = []
    Ca = 5
    x = 1
    v_eq::Float64 = 0.0
    Ca_eq::Float64 = 0.0
    x_eq::Float64 = 0.0
    try
        v_eq, Ca_eq, x_eq = Ca_x_eq(p, min(sol(sol.t, idxs=(6))...))
    catch e
        if debug
            print("No equilibrium found.")
        end
        return [0]
    end

    for i in 2:length(sol)
        x = 1
        Ca = 5
        if sol[i-1][x] < x_eq && sol[i][x] < x_eq && sol[i][Ca] <= Ca_eq < sol[i-1][Ca]
            push!(resets, i)
        end
    end

    if debug
        print("$(length(resets)) burst resets observed.")
    end

    V_threshold = 0.0

    # Obtain spike counts per burst.
    spike_counts = []
    for i in 1:length(resets)-1
        spike_count = 0
        for j in resets[i]:resets[i+1]
            if sol[j-1][6] < V_threshold < sol[j][6]
                spike_count += 1
            end
        end
        if spike_count > 0
            push!(spike_counts, spike_count)
        end
    end

    if length(spike_counts) < 2
        return [0]
    else
        return spike_counts[2:end]
    end
end

In [ ]:
function transitionMap(spike_counts)
    plt = scatter(1, markeralpha=0.2, legend=false, aspect_ratio=:equal, size=(600, 600), xticks=0:maximum(spike_counts), yticks=0:maximum(spike_counts), xlims=(-0.5, maximum(spike_counts) + 0.5), ylims=(-0.5, maximum(spike_counts) + 0.5))
    plot!(plt, [0, maximum(spike_counts)], [0, maximum(spike_counts)], linealpha=0.2)
    for i in 1:length(spike_counts)-1
        push!(plt, (spike_counts[i], spike_counts[i+1]))
    end
    return plt
end

In [ ]:
function markovChain(spike_counts)
    if length(spike_counts) == 0
        return zeros(0, 0)
    end
    size = max(spike_counts...)
    chain = zeros(size, size)
    for i in 1:length(spike_counts)-1
        chain[spike_counts[i], spike_counts[i+1]] += 1
    end
    for row in 1:size
        if max(chain[row, :]...) == 0.0
            # If we don't know what comes next, we consider all
            # outcomes equiprobable.
            chain[row, :] = ones(size)
        end
        # Normalize rows to have total probability 1.
        chain[row, :] = normalize!(chain[row, :], 1)
    end

    return chain
end

# Generating a spike count sweep
Specify overall sweep dimensions:
- ΔCa_min: Minimum calcium shift.
- ΔCa_max: Maximum calcium shift.
- ΔCa_resolution: Number of ΔCa values to sweep over.
- Δx_min: Minimum x shift.
- Δx_max: Maximum x shift.
- Δx_resolution: Number of Δx values to sweep over.
- chunk_proportion: Number of chunks to split the sweep into per dimension. The total number of chunks will be `1/chunk_proportion^2`.

In [ ]:
ΔCa_min = -60.0
ΔCa_max = -35.0 #50.0
ΔCa_resolution = 50
Δx_min = -1.5
Δx_max = 0.0 #3.0
Δx_resolution = Int(ΔCa_resolution/2)
chunk_proportion = 1/25

tspan = (0, 1.0f5)

In [14]:
for chunk in 1:Int(1/chunk_proportion)^2
    params = []
    chunk_ΔCa_min = ΔCa_min + (ΔCa_max - ΔCa_min)*chunk_proportion*trunc(Int, chunk*chunk_proportion)
    chunk_ΔCa_max = ΔCa_min + (ΔCa_max - ΔCa_min)*(chunk_proportion*(trunc(Int, chunk*chunk_proportion)+1)-1/ΔCa_resolution)
    for ΔCa in range(chunk_ΔCa_min, chunk_ΔCa_max, length=Int(ΔCa_resolution*chunk_proportion))
        chunk_Δx_min = Δx_min + (Δx_max - Δx_min)*chunk_proportion*(chunk%(1/chunk_proportion)-1)
        chunk_Δx_max = Δx_min + (Δx_max - Δx_min)*(chunk_proportion*(chunk%(1/chunk_proportion)) - 1/Δx_resolution)
        for Δx in range(chunk_Δx_min, chunk_Δx_max, length=Int(Δx_resolution*chunk_proportion))
            p = GPUPlant.default_params + @SVector Float32[
                0.0e0, # Cₘ
                0.0e0, # gI
                0.0e0, # gK
                0.0e0, # gₕ
                0.0e0, # gL
                0.0e0, # gT
                0.0e0, # gKCa
                0.0e0, # EI
                0.0e0, # EK
                0.0e0, # Eₕ
                0.0e0, # EL
                0.0e0, # ECa
                0.0e0, # Kc
                0.0e0, # τₓ
                0.0e0, # ρ
                Δx,    # Δx
                ΔCa    # ΔCa
            ]
            push!(params, p)
        end
        ranges = Dict(
            "ΔCa_min" => chunk_ΔCa_min,
            "ΔCa_max" => chunk_ΔCa_max,
            "Δx_min" => chunk_Δx_min,
            "Δx_max" => chunk_Δx_max
        )
        @save "output/chunk_$(chunk)_ranges.jld2" ranges
    end
    function initial_conditions(p)
        try
            v_eq, Ca_eq, x_eq = Ca_x_eq(p)
            u0 = @SVector Float32[x_eq, state.y, state.n, state.h, Ca_eq-0.2, v_eq, state.Isyn]
        catch e
            u0 = state
        end
        return u0
    end
    prob = ODEProblem{false}(GPUPlant.melibeNew, u0, tspan, params[1])
    prob_func(prob, i, repeat) = remake(prob, u0=initial_conditions(params[trunc(Int, i)]), p=params[trunc(Int, i)]) # Why are we getting Floats here?
    function output_func(sol, i)
        spike_counts = countSpikes(sol, params[i])
        if length(spike_counts) < 2
            return (0, false)
        else
            return (norm(markovChain(spike_counts)), false)
        end
    end
    monteprob = EnsembleProblem(prob, output_func=output_func, prob_func=prob_func, safetycopy=false)
    sol = solve(monteprob, GPUTsit5(), EnsembleGPUKernel(), trajectories=trunc(Int, ΔCa_resolution*Δx_resolution*chunk_proportion^2), adaptive=false, dt=1f-1, verbose=false);
    @save "output/chunk_$(chunk).jld2" sol
    println("Finished chunk $chunk of $(Int(1/chunk_proportion)^2): $(round(100*chunk*chunk_proportion^2, digits=2))%")
end

In [12]:
@load "output/chunk_366_ranges.jld2" ranges
@load "output/chunk_366.jld2" sol
ranges

Dict{String, Float64} with 4 entries:
  "ΔCa_max" => 0.9575
  "ΔCa_min" => -2.4
  "Δx_min"  => -1.0
  "Δx_max"  => -0.9025

In [12]:
plt = heatmap(
    xlabel="\$\\Delta_{Ca}\$",
    ylabel="\$\\Delta_x\$",
    #xlim=(-40, -30),
    #ylim=(-0.75, -0.5),
    title="Spikes per burst: \$\\ln(1+\\sigma^2)\$",
    color=:thermal,
    size=(1000, 750),
    dpi=1000
);

In [13]:
for i in 1:(Int(1/chunk_proportion)^2-1)
    @load "output/chunk_$(i)_ranges.jld2" ranges
    @load "output/chunk_$(i).jld2" sol
    heatmap!(
        plt,
        range(ranges["ΔCa_min"], ranges["ΔCa_max"], length=Int(ΔCa_resolution*chunk_proportion)),
        range(ranges["Δx_min"], ranges["Δx_max"], length=Int(Δx_resolution*chunk_proportion)),
        reshape(sol.u, Int(Δx_resolution*chunk_proportion), Int(ΔCa_resolution*chunk_proportion)),
    )
end
plt

In [33]:
# Save `plt` to a file locally with full resolution.
savefig(plt, "heatmap.png")

In [15]:
p = Plant.Params(
    ΔCa = range_ΔCa[5],
    Δx = range_Δx[4]
)

voltages = range(-70, 20, length=1000)

# Here we let y be at its equilibrium.
IKCa = [p.gI*Plant.h∞(v)*Plant.m∞(v)^3*(p.EI-v) + p.gK*Plant.n∞(v)^4*(p.EK-v) + p.gT*Plant.x∞(p, v)*(p.EI-v) + p.gₕ*(p.Eₕ-v)/((1+exp(10*(v-50)))*(1+exp(-(v-63)/7.8))^3) + p.gL*(p.EL-v) for v in voltages]

x_null_Cas = [0.5*IKCa[i]/(p.gKCa*(voltages[i]-p.EK) - IKCa[i]) for i in 1:length(voltages)]
x_null_xs = [Plant.x∞(p, v) for v in voltages]

# The paper says we should use these, but they don't really seem to work.
# IT = [p.gI*Plant.h∞(voltages[i])*Plant.m∞(voltages[i])^3*(p.EI-voltages[i]) + p.gK*Plant.n∞(voltages[i])^4*(p.EK-voltages[i]) - IKCa[i] + p.gₕ*(p.Eₕ-voltages[i])/((1+exp(10*(voltages[i]-50)))*(1+exp(-(voltages[i]-63)/7.8))^3) + p.gL*(p.EL-voltages[i]) for i in 1:length(voltages)]
# Ca_null_Cas = [IT[i]*(p.ECa - voltages[i] + p.ΔCa)/(p.EI-voltages[i]) for i in 1:length(voltages)]
Ca_null_Cas = [p.Kc*Plant.x∞(p, v)*(p.ECa-v+p.ΔCa) for v in voltages]
Ca_null_xs = [Plant.x∞(p, v) for v in voltages]


plt = plot(x_null_Cas, x_null_xs, xlabel="Ca", ylabel="x", xlim=(0.0, 1.5), ylim=(0.0, 0.95), legend=false)
plot!(plt, Ca_null_Cas, Ca_null_xs)

long_tspan = (0, 1e6)
prob = ODEProblem(Plant.melibeNew!, u0, long_tspan, p)
sol = solve(prob, Tsit5(), reltol=1e-5, abstol=1e-5)

v_eq, Ca_eq, x_eq = Ca_x_eq(p, min(sol(sol.t, idxs=(6))...))

burst_resets = []
for i in 2:length(sol)
    x = 1
    Ca = 5
    if sol[i-1][x] < x_eq && sol[i][x] < x_eq && sol[i][Ca] <= Ca_eq < sol[i-1][Ca]
        push!(burst_resets, i)
    end
end

plot!(plt, sol, idxs=(5, 1), lw=0.2)
scatter!(plt, [Ca_eq], [x_eq], ms=2, mc=:black)
scatter!(plt, [sol[i][5] for i in burst_resets], [sol[i][1] for i in burst_resets], ms=1.0, mc=:black)

LoadError: UndefVarError: range_ΔCa not defined